In [1]:
from pathlib import Path
import os

yr = 2023
d = 11

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  import numpy as np

  arr = np.array([list(x) for x in inp.split('\n')], dtype='<U4')
  cnt = 1
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      if arr[i,j] == '#':
        arr[i,j] = str(cnt)
        cnt += 1
  return arr

In [3]:
def get_all_distances_expand(formatted_input, expand_num=2):
  from tqdm.notebook import tqdm
  from itertools import combinations
  import numpy as np

  def manhattan_distance(p1, p2):
    return np.sum(np.abs(np.array(p1)-np.array(p2)))

  def get_coords_from_num(num):
    return list(zip(*np.where(formatted_input == num)))[0]


  rows_to_expand = [i for i in range(formatted_input.shape[0])
  if all([x=='.' for x in formatted_input[i]])]
  columns_to_expand = [i for i in range(formatted_input.shape[1])
  if all([x=='.' for x in formatted_input.T[i]])]

  def expand_locs(l1, l2):
    row_locs = [l1[0], l2[0]]
    col_locs = [l1[1], l2[1]]
    row_expand_sum = (sum([int(min(row_locs) < r and r < max(row_locs))
     for r in rows_to_expand]) * (expand_num-1))
    col_expand_sum = (sum([int(min(col_locs) < r and r < max(col_locs))
     for r in columns_to_expand]) * (expand_num-1))

    row_locs[row_locs.index(max(row_locs))] = max(row_locs) + row_expand_sum
    col_locs[col_locs.index(max(col_locs))] = max(col_locs) + col_expand_sum

    return (row_locs[0], col_locs[0]), (row_locs[1], col_locs[1])


  m = max([int(x) for x in formatted_input.flatten() if x.isnumeric()])
  pairs = combinations(list(range(1,m+1)), 2)
  s = np.array([0], dtype=np.ulonglong) # This will probably get pretty big
  for p in tqdm(list(pairs)):
    l1 = get_coords_from_num(str(p[0]))
    l2 = get_coords_from_num(str(p[1]))
    l1, l2 = expand_locs(get_coords_from_num(str(p[0])),
                         get_coords_from_num(str(p[1])))
    s[0] += manhattan_distance(l1, l2)
  return s[0]

In [4]:
import time

t = time.time()

formatted_input = format_input(inp)

print(get_all_distances_expand(formatted_input, expand_num=2))
print(get_all_distances_expand(formatted_input, expand_num=1000000))

print('\nRUNTIME: ', time.time()-t)

  0%|          | 0/106030 [00:00<?, ?it/s]

10490062


  0%|          | 0/106030 [00:00<?, ?it/s]

382979724122

RUNTIME:  60.19018769264221
